Nesse notebook, começo a testar outros algoritmos e faço processo de tunning nos parâmetros do modelo.
A ideia aqui é chegar em melhores métricas de avaliação de modelo.

In [1]:
import pandas as pd
from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score, average_precision_score
from skopt import forest_minimize

from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv("fully_annoteded.csv").dropna(subset=['y'])
df.head()

,title,y,upload_date,view_count,tempo_desde_pub
0,"""Kaggle: command not found fix"" for Mac Users",0,30/11/2020,216,275
1,#1 - Data Science na PrÃ¡tica - Conhecendo o K...,0,03/08/2020,808,394
2,"#AndroidDevChallenge - Helpful innovation, pow...",0,22/06/2020,5779436,436
3,#Data Science SQL Full Outer join Keyword | Ek...,0,28/08/2021,1,4
4,#Data Science SQL Left join Keyword | Ekasclou...,0,28/08/2021,0,4


In [3]:
df.duplicated().mean()

0.0

In [4]:
df.duplicated(['title']).mean()

0.0

In [5]:
df.shape

(1182, 5)

In [6]:
df['view_per_day'] = round(df['view_count'] / df['tempo_desde_pub'], 4)
df = df.drop(['tempo_desde_pub'], axis=1)
df.head()

,title,y,upload_date,view_count,view_per_day
0,"""Kaggle: command not found fix"" for Mac Users",0,30/11/2020,216,0.7855
1,#1 - Data Science na PrÃ¡tica - Conhecendo o K...,0,03/08/2020,808,2.0508
2,"#AndroidDevChallenge - Helpful innovation, pow...",0,22/06/2020,5779436,13255.5872
3,#Data Science SQL Full Outer join Keyword | Ek...,0,28/08/2021,1,0.2500
4,#Data Science SQL Left join Keyword | Ekasclou...,0,28/08/2021,0,0.0000


### Random Forest Classifier

Para esse algoritmo, testei diferentes valores para os seguintes parâmetros:
* n_estimators
* min_samples_leaf
* class_weight
* n_jobs

Além de testar variações do parâmetro de ngram no TFIDF

In [7]:
X = df.copy().drop(['y', 'upload_date'], axis=1)
y = df['y']

In [8]:
Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size=0.55, random_state=0)
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((531, 3), (651, 3), (531,), (651,))

In [9]:
title_train = Xtrain['title']
title_val = Xval['title']

#min_df = minimo de exemplos de palavra
#ngram_range = numero de unigramas
title_vec = TfidfVectorizer(min_df=1, ngram_range=(1,1))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [10]:
Xtrain_wtitle = hstack([Xtrain.drop(['title'], axis=1), title_bow_train])
Xval_wtitle = hstack([Xval.drop(['title'], axis=1), title_bow_val])

Xtrain_wtitle.shape, Xval_wtitle.shape

((531, 1529), (651, 1529))

In [11]:
mdl = RandomForestClassifier(n_estimators=100, random_state=0, min_samples_leaf=1, class_weight='balanced', n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_jobs=6, random_state=0)

In [12]:
p = mdl.predict_proba(Xval_wtitle)[ : , 1]

In [13]:
print('precision: {} and roc: {}'.format(average_precision_score(yval, p), roc_auc_score(yval, p)))

precision: 0.32113680450630394 and roc: 0.7318334350213545


precision: 0.32113680450630394 and roc: 0.7318334350213545 -> unigrama
precision: 0.31808388628023254 and roc: 0.7176479560707748 -> bigrama

### Testando o LightGBM

Utilizei a função *forest_minimize* da biblioteca *skopt* para chegar nos melhores parâmetros para o modelo LGBM.
Utilizei esse método pois ele é melhor que o GridSearch, já que este procura de forma aleatória e tenta prever o erro, para o próximo 'loop' ser melhor que o anterior.

In [14]:
mdl2 = LGBMClassifier(random_state=0, class_weight='balanced', n_jobs=6)
mdl2.fit(Xtrain_wtitle, ytrain)

LGBMClassifier(class_weight='balanced', n_jobs=6, random_state=0)

In [15]:
p2 = mdl2.predict_proba(Xval_wtitle)[ : , 1]

D:\Arquivos de Programas\Anaconda\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Métricas de avaliação sem tunning:

In [16]:
print('precision: {} and roc: {}'.format(average_precision_score(yval, p2), roc_auc_score(yval, p2)))

precision: 0.12864127819187215 and roc: 0.5719188529591214


Criação da função para otimização, aqui a ideia é chegar numa melhor **precisão**, mas tambem acompanhar o **ROC**

In [17]:
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    min_df = params[6]
    ngram_range = (1, params[7])

    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    title_bow_train = title_vec.fit_transform(title_train)
    title_bow_val = title_vec.transform(title_val)

    Xtrain_wtitle = hstack([Xtrain.drop(['title'], axis=1), title_bow_train])
    Xval_wtitle = hstack([Xval.drop(['title'], axis=1), title_bow_val])

    mdl3 = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth,
                          min_child_samples=min_child_samples, subsample=subsample,
                          colsample_bytree=colsample_bytree, bagging_freq=1, n_estimators=n_estimators, random_state=0,
                          class_weight="balanced", n_jobs=6)

    mdl3.fit(Xtrain_wtitle, ytrain)

    p = mdl3.predict_proba(Xval_wtitle)[ : , 1]
    print(roc_auc_score(yval, p))

    return -average_precision_score(yval, p)

In [18]:
space = [(1e-3, 1e-1, 'log-uniform'), # lr
         (1, 10),                      # max_depth
         (1, 20),                      # min_child_samples
         (0.05, 1.),                   # subsample
         (0.05, 1.),                   # colsample_bytree
         (100, 1000),                  # n_estimators
         (1, 5),                       # mind_df
         (1, 5)]                       # ngram_range

In [19]:
res = forest_minimize(tune_lgbm, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272, 3, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


D:\Arquivos de Programas\Anaconda\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
D:\Arquivos de Programas\Anaconda\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.597895057962172
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.2170
Function value obtained: -0.1500
Current minimum: -0.1500
Iteration No: 2 started. Evaluating function at random point.
[0.053887464791860025, 1, 15, 0.7437489153990157, 0.8675167974293533, 549, 3, 4]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.5827181208053691
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.1490
Function value obtained: -0.1089
Current minimum: -0.1500
Iteration No: 3 started. Evaluating function at random point.
[0.004151454520895999, 6, 20, 0.8682075103820793, 0.9491436163200662, 411, 4, 3]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.5854636973764491
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.1780
Function value obtained: -0.1263
Current minimum: -0.1500
Iteration No: 4 started. Evaluating functio

D:\Arquivos de Programas\Anaconda\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
D:\Arquivos de Programas\Anaconda\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
D:\Arquivos de Programas\Anaconda\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.5910158633312995
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.4860
Function value obtained: -0.1434
Current minimum: -0.1500
Iteration No: 5 started. Evaluating function at random point.
[0.08530558241838007, 8, 19, 0.2137736299768322, 0.1313765544201984, 961, 4, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.5801250762660158
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.1510
Function value obtained: -0.1090
Current minimum: -0.1500
Iteration No: 6 started. Evaluating function at random point.
[0.003567949451535685, 10, 19, 0.7232951768944309, 0.7298538828427115, 939, 4, 3]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


...

0.67477120195241
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 1.3650
Function value obtained: -0.2561
Current minimum: -0.2785
Iteration No: 49 started. Searching for the next optimal point.
[0.07473498545892235, 9, 1, 0.8020499855076734, 0.10705426389117481, 562, 1, 2]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


D:\Arquivos de Programas\Anaconda\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.6855399633923124
Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 1.4480
Function value obtained: -0.2934
Current minimum: -0.2934
Iteration No: 50 started. Searching for the next optimal point.
[0.09543054224256677, 8, 1, 0.9453999830880128, 0.26181401391730624, 589, 1, 4]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


D:\Arquivos de Programas\Anaconda\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.6219798657718121
Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 1.8030
Function value obtained: -0.2379
Current minimum: -0.2934


Ao selecionar o atributo **x** após a aplicação da busca de melhores parâmetros, temos o melhor valor de cada parâmetro que decidi tunar.

In [20]:
print("""
lr: {}
max_depth: {}
min_child_samples: {}
subsample: {}
colsample_bytree: {}
n_estimators: {}
min_df: {}
ngram_range: {}
""".format( res.x[0], res.x[1], res.x[2], res.x[3], res.x[4], res.x[5], res.x[6], res.x[7]))


lr: 0.07473498545892235
max_depth: 9
min_child_samples: 1
subsample: 0.8020499855076734
colsample_bytree: 0.10705426389117481
n_estimators: 562
min_df: 1
ngram_range: 2



O atributo **fun** mostra qual a melhor métrica que o modelo conseguiu chegar.

In [21]:
res.fun

-0.293430888435203

### Logistic Regression

Nas próximas células, testo a aplicação de uma regressão logistica, alterando os tipos de scaler (já que esse algoritmo é sensível à amplitude dos dados)

In [22]:
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix

In [29]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

#scaler = StandardScaler()
#Xtrain_wtitle2[: , :2] = scaler.fit_transform(Xtrain_wtitle2[: , :2].todense())
#Xval_wtitle2[: , :2] = scaler.fit_transform(Xval_wtitle2[: , :2].todense())

scaler = MaxAbsScaler()
Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
Xval_wtitle2 = scaler.fit_transform(Xval_wtitle2)

In [30]:
Xval_wtitle2.shape

(651, 1529)

In [71]:
mdl3 = LogisticRegression(C=1.5, n_jobs=6, random_state=0)
mdl3.fit(Xtrain_wtitle2, ytrain)

LogisticRegression(C=1.5, n_jobs=6, random_state=0)

In [72]:
p = mdl3.predict_proba(Xval_wtitle2)[: , 1]

In [73]:
average_precision_score(yval, p)

0.28445728618896904

In [74]:
roc_auc_score(yval, p)

0.7430750457596095

standard_scaler = acc: 0.2641978963609311 / roc: 0.7303233679072605
abs_scaler = acc: 0.2828859479124313 / roc: 0.7421598535692494
abs_scaler + C=1.5 = acc: 0.28294177477226495 / roc: 0.7429225137278829